In [2]:
#do your imports
from census import Census
from us import states
import pandas as pd

In [3]:
#define the API key -> every user needs their own
c = Census("495a349ce22bdb1294b378fb199e4f27e57471a9")

In [9]:
#Median age
medAge = c.acs.state_county_blockgroup(fields = ('NAME', 'B01002_001E'), state_fips = states.SC.fips, county_fips="*", tract = "*", blockgroup = "*")
medAge = pd.DataFrame(medAge)
medAge

,NAME,B01002_001E,state,county,tract,block group
0,"Block Group 2, Census Tract 20.07, Charleston ...",45.0,45,019,002007,2
1,"Block Group 3, Census Tract 20.08, Charleston ...",46.8,45,019,002008,3
2,"Block Group 3, Census Tract 20.09, Charleston ...",49.1,45,019,002009,3
3,"Block Group 3, Census Tract 21.03, Charleston ...",52.2,45,019,002103,3
4,"Block Group 2, Census Tract 21.04, Charleston ...",66.1,45,019,002104,2
...,...,...,...,...,...,...
3403,"Block Group 3, Census Tract 601.02, York Count...",32.9,45,091,060102,3
3404,"Block Group 1, Census Tract 602, York County, ...",36.5,45,091,060200,1
3405,"Block Group 3, Census Tract 602, York County, ...",42.9,45,091,060200,3
3406,"Block Group 4, Census Tract 602, York County, ...",41.7,45,091,060200,4


In [10]:
#Less than five years
under5 = c.acs.state_county_blockgroup(fields = ('NAME','B01001_027E', 'B01001_003E', 'B01001_001E' ), state_fips = states.SC.fips, county_fips="*", tract = "*", blockgroup = "*")
pd_under5 = pd.DataFrame(under5)
pd_under5.rename(columns={"B01001_027E": "FiveYearsAgeFemBG", "B01001_003E": "FiveYearsAgeMaleBG", "B01001_001E" : "TotalPopulationBG"}, inplace=True)
pd_under5

,NAME,FiveYearsAgeFemBG,FiveYearsAgeMaleBG,TotalPopulationBG,state,county,tract,block group
0,"Block Group 2, Census Tract 20.07, Charleston ...",103.0,47.0,2300.0,45,019,002007,2
1,"Block Group 3, Census Tract 20.08, Charleston ...",0.0,0.0,1061.0,45,019,002008,3
2,"Block Group 3, Census Tract 20.09, Charleston ...",0.0,0.0,1097.0,45,019,002009,3
3,"Block Group 3, Census Tract 21.03, Charleston ...",0.0,0.0,263.0,45,019,002103,3
4,"Block Group 2, Census Tract 21.04, Charleston ...",0.0,0.0,530.0,45,019,002104,2
...,...,...,...,...,...,...,...,...
3403,"Block Group 3, Census Tract 601.02, York Count...",28.0,6.0,993.0,45,091,060102,3
3404,"Block Group 1, Census Tract 602, York County, ...",26.0,52.0,1020.0,45,091,060200,1
3405,"Block Group 3, Census Tract 602, York County, ...",0.0,41.0,1178.0,45,091,060200,3
3406,"Block Group 4, Census Tract 602, York County, ...",7.0,47.0,1032.0,45,091,060200,4


In [75]:
svi = pd_under5.copy()

In [76]:
def zscore(column):
    return (column - column.mean()) / column.std()

def minmax_normalize(column):
    return (column - column.min()) / (column.max() - column.min())

def maxmin_normalize(column):
    return (column - column.max()) / (column.max() - column.min())


In [77]:
svi = pd_under5.copy()
svi_fiat = pd.DataFrame()
#svi_fiat['NAME'] = svi['NAME']
vulnerability_features = ["FiveYearsAgeFemBG", "FiveYearsAgeMaleBG", "TotalPopulationBG"]

for column in vulnerability_features:
    #svi[column+'_score']       = zscore(svi[column])

    svi_fiat[column] = minmax_normalize(svi[column])

    #svi[column+'_maxmin_norm'] = maxmin_normalize(svi[column]).multiply(-1)



In [78]:
svi_fiat

,FiveYearsAgeFemBG,FiveYearsAgeMaleBG,TotalPopulationBG
0,0.157492,0.072868,0.189659
1,0.000000,0.000000,0.087491
2,0.000000,0.000000,0.090459
3,0.000000,0.000000,0.021687
4,0.000000,0.000000,0.043704
...,...,...,...
3403,0.042813,0.009302,0.081883
3404,0.039755,0.080620,0.084110
3405,0.000000,0.063566,0.097139
3406,0.010703,0.072868,0.085099


In [106]:
dict_var     = ['FiveYearsAgeFemBG','FiveYearsAgeMaleBG']
#scores_vars  = [0.5,0.5]

def ranking(df, group_name, dict_var, scores_vars):
    if not scores_vars:
        scores_vars = [1/len(dict_var)]*len(dict_var)

    df[group_name] = 0
    for column, factor in zip(dict_var, scores_vars):
        df[group_name] = df[group_name] + df[column]*factor

    return scores_vars

scores_vars_test = ranking(svi_fiat,'cathegory1', dict_var, scores_vars)


In [107]:
scores_vars_test

[0.5, 0.5]

0.5

In [73]:
svi_fiat['score_group1'] = svi_fiat['FiveYearsAgeFemBG']*0.5 + svi_fiat['FiveYearsAgeMaleBG']*0.5 
svi_fiat['score_group2'] = svi_fiat['TotalPopulationBG']

In [74]:
svi_fiat

,FiveYearsAgeFemBG,FiveYearsAgeMaleBG,TotalPopulationBG,score_group1,score_group2
0,0.157492,0.072868,0.189659,0.115180,0.189659
1,0.000000,0.000000,0.087491,0.000000,0.087491
2,0.000000,0.000000,0.090459,0.000000,0.090459
3,0.000000,0.000000,0.021687,0.000000,0.021687
4,0.000000,0.000000,0.043704,0.000000,0.043704
...,...,...,...,...,...
3403,0.042813,0.009302,0.081883,0.026058,0.081883
3404,0.039755,0.080620,0.084110,0.060188,0.084110
3405,0.000000,0.063566,0.097139,0.031783,0.097139
3406,0.010703,0.072868,0.085099,0.041786,0.085099


In [69]:
def sum(df, column1, column2, column3):
    sum_df = pd.DataFrame({
        'Sum': [df[column1].sum(), df[column2].sum(), df[column3].sum()]
    }, index=[column1, column2, column3])

    return sum_df

In [70]:
sum_svi_fiat = sum(svi_fiat,"FiveYearsAgeFemBG", "FiveYearsAgeMaleBG", "TotalPopulationBG")